In [50]:
import pandas as pd
ratings = pd.read_csv('ratings.dat', sep='::', engine = 'python', header=None)
ratings.columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']
movies = pd.read_csv('movies.dat', sep='::', engine = 'python',
                     encoding="ISO-8859-1", header = None)
movies.columns = ['MovieID', 'Title', 'Genres']

In [54]:
movies_head = movies.copy()
movies_head.drop('Genres', axis=1, inplace=True)
movies_head.head(20)

,MovieID,Title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
5,6,Heat (1995)
6,7,Sabrina (1995)
7,8,Tom and Huck (1995)
8,9,Sudden Death (1995)
9,10,GoldenEye (1995)


In [35]:
ratings

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [28]:
# Get set of unique genres
genres = set()
for movie in movies['Genres']:
    genres.update(movie.split('|'))
genres

{'Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

In [29]:
rating_merged = ratings.merge(movies, left_on = 'MovieID', right_on = 'MovieID')
rating_merged

,UserID,MovieID,Rating,Timestamp,Title,Genres
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
1000204,5949,2198,5,958846401,Modulations (1998),Documentary
1000205,5675,2703,3,976029116,Broken Vessels (1998),Drama
1000206,5780,2845,1,958153068,White Boys (1999),Drama
1000207,5851,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western


In [37]:
# Find most popular movie in given genre, return movie id and title

def most_popular(genre):
    genre_ratings = rating_merged[rating_merged['Genres'].str.contains(genre)]
    id = genre_ratings.groupby('MovieID').size().sort_values(ascending=False)[:10]
    return movies[movies['MovieID'].isin(id.index)][['MovieID', 'Title']]

most_popular("Action")

# Save most popular movies in each genre to one json file
import json
genre_dict = {}
for genre in genres:
    genre_dict[genre] = most_popular(genre).to_dict('records')
    
with open('most_popular.json', 'w') as outfile:
    json.dump(genre_dict, outfile)


# Read json file and print most popular movies in each genre
with open('most_popular.json') as json_file:
    data = json.load(json_file)
    for genre in data:
        print(genre)
        for movie in data[genre]:
            print(movie['Title'])
        print("\n")